In [2]:
import os

import fsspec
import hydra
import lightning as L
import omegaconf
import rich.syntax
import rich.tree
import torch

import dataloader
import diffusion
import utils

import json
import mauve

omegaconf.OmegaConf.register_new_resolver(
  'cwd', os.getcwd)
omegaconf.OmegaConf.register_new_resolver(
  'device_count', torch.cuda.device_count)
omegaconf.OmegaConf.register_new_resolver(
  'eval', eval)
omegaconf.OmegaConf.register_new_resolver(
  'div_up', lambda x, y: (x + y - 1) // y)

'/home/lucibello'

In [11]:
HOME = os.environ.get('HOME')
checkpoint_path=f"{HOME}/Git/remdm/outputs/checkpoints/mdlm.ckpt"
T=0
sampling_steps=128 #1024
p=0.9
num_sample_batches=1 # 5000
global_batch_size=512
devices=1
generated_seqs_path=f"{HOME}/Git/remdm/outputs/mdlm_T-{sampling_steps}_topp-{p}.json"


In [14]:
# read config file from configs/config.yaml

config = omegaconf.OmegaConf.load("configs/config.yaml")

with hydra.initialize(config_path="configs/", version_base=None):
    config = hydra.compose(config_name="config", overrides=[
        f"data=openwebtext-split",
        f"eval.checkpoint_path={checkpoint_path}",
        f"time_conditioning=false",
        f"T={T}",
        f"loader.global_batch_size={global_batch_size}",
        f"sampling.steps={sampling_steps}",
        f"seed=1",
        f"loader.batch_size=1",
        f"loader.eval_batch_size=1",
        f"eval.perplexity_batch_size=1",
        f"sampling.num_sample_batches={num_sample_batches}",
        f"sampling.generated_seqs_path={generated_seqs_path}",
        f"sampling.nucleus_p={p}",
        f"sampling.sampler=mdlm",
        f"trainer.devices={devices}",
        f"data.cache_dir={HOME}/Git/remdm/",
    ])
tokenizer = dataloader.get_tokenizer(config)
_, valid_loader = dataloader.get_dataloaders(config, tokenizer, valid_seed=config.seed, skip_train=True)


/home/lucibello/miniconda3/envs/remdm/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using 1 GPUs for training.
Using 1 batch size and 1 eval batch size. Global batch size is 512.
num_nodes: 1,        accumulate_grad_batches: 512


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [19]:
config.data.valid, config.data.wrap, config.data.cache_dir

('openwebtext-valid', True, '/home/lucibello/Git/remdm/')

In [16]:
valid_set = dataloader.get_dataset(
      config.data.valid,
      tokenizer,
      wrap=config.data.wrap,
      mode="validation",
      cache_dir=config.data.cache_dir,
      block_size=config.model.length,
      streaming=False)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte